### 문제 5-1 : 카페 메뉴 도구(Tool) 호출 체인 구현

In [8]:
import os
import warnings
warnings.filterwarnings('ignore')

from langchain.tools import tool
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.tools import TavilySearchResults
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.schema import Document
from langchain_core.runnables import chain
import json

#### 1. 카페 메뉴 데이터 파일 생성 및 벡터 DB 구축

In [9]:
# 기존 파일 경로 확인
menu_file_path = "../data/cafe_menu_data.txt"

# 파일 존재 여부 확인
if os.path.exists(menu_file_path):
    print(f"기존 카페 메뉴 파일 발견: {menu_file_path}")
    
    # 파일 내용 미리보기
    with open(menu_file_path, "r", encoding="utf-8") as f:
        content = f.read()
        print(f"파일 크기: {len(content)} 문자")
        print("파일 내용 미리보기:")
        print(content[:200] + "..." if len(content) > 200 else content)
else:
    print(f"파일을 찾을 수 없습니다: {menu_file_path}")
    print("파일 경로를 확인해주세요.")
    
def create_vector_db():
    """카페 메뉴 텍스트를 벡터 DB로 변환"""
    
    # 기존 파일 경로 사용
    menu_file_path = "../data/cafe_menu_data.txt"
    
    # 텍스트 로더로 파일 읽기
    loader = TextLoader(menu_file_path, encoding="utf-8")
    documents = loader.load()
    
    # 텍스트 분할기 설정
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,  # 각 청크의 크기
        chunk_overlap=50,  # 청크 간 겹치는 부분
        separators=["\n\n", "\n", ".", " "]
    )
    
    # 문서 분할
    split_docs = text_splitter.split_documents(documents)
    
    print(f"문서가 {len(split_docs)}개의 청크로 분할되었습니다.")
    
    # Ollama 임베딩 모델 설정
    embeddings = OllamaEmbeddings(
        model="bge-m3",
        base_url="http://localhost:11434"
    )
    
    # FAISS 벡터 스토어 생성
    print("벡터 DB 생성 중...")
    vectorstore = FAISS.from_documents(split_docs, embeddings)
    
    # 벡터 DB 저장
    os.makedirs("./db", exist_ok=True)
    vectorstore.save_local("./db/cafe_db")
    
    print("벡터 DB 생성 및 저장 완료: ./db/cafe_db")
    return vectorstore

# 벡터 DB 생성 실행
try:
    vectorstore = create_vector_db()
    print("벡터 DB 구축 성공")
except Exception as e:
    print(f"벡터 DB 구축 실패: {e}")


기존 카페 메뉴 파일 발견: ../data/cafe_menu_data.txt
파일 크기: 1681 문자
파일 내용 미리보기:
1. 아메리카노
   • 가격: ₩4,500
   • 주요 원료: 에스프레소, 뜨거운 물
   • 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다.

2. 카페라떼
   • 가격: ₩5,500
   • 주요 원료: 에스프레...
문서가 10개의 청크로 분할되었습니다.
벡터 DB 생성 중...
벡터 DB 생성 및 저장 완료: ./db/cafe_db
벡터 DB 구축 성공


#### 2. 3개의 도구를 정의하고 LLM에 바인딩

In [10]:
# 1) 웹 검색 도구 - tavily_search_func
@tool
def tavily_search_func(query: str) -> str:
    """웹에서 최신 정보를 검색합니다. 최신 커피 트렌드, 카페 위치 정보 등에 활용하세요."""
    try:
        search = TavilySearchResults(max_results=3)
        results = search.invoke(query)
        
        # 검색 결과를 문자열로 정리
        formatted_results = []
        for result in results:
            if isinstance(result, dict):
                title = result.get('title', '제목 없음')
                content = result.get('content', '내용 없음')
                formatted_results.append(f"제목: {title}\n내용: {content}\n")
        
        return "\n".join(formatted_results) if formatted_results else "검색 결과를 찾을 수 없습니다."
        
    except Exception as e:
        return f"웹 검색 중 오류가 발생했습니다: {str(e)}"

# 2) 위키피디아 검색 도구 - wiki_summary
@tool
def wiki_summary(topic: str) -> str:
    """위키피디아에서 일반 지식을 검색하고 요약합니다. 커피 역사, 음료 제조 방법 등에 활용하세요."""
    try:
        wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
        result = wikipedia.run(topic)
        
        # 결과가 너무 길면 앞부분만 반환
        if len(result) > 1000:
            result = result[:1000] + "..."
            
        return result
        
    except Exception as e:
        return f"위키피디아 검색 중 오류가 발생했습니다: {str(e)}"

# 3) 로컬 DB 검색 도구 - db_search_cafe_func
@tool
def db_search_cafe_func(query: str) -> str:
    """로컬 카페 메뉴 DB에서 정보를 검색합니다. 특정 메뉴의 가격, 재료, 설명을 찾을 때 사용하세요."""
    try:
        # 임베딩 모델 로드
        embeddings = OllamaEmbeddings(
            model="bge-m3",
            base_url="http://localhost:11434"
        )
        
        # 기존 벡터 DB 로드
        vectorstore = FAISS.load_local("./db/cafe_db", embeddings, allow_dangerous_deserialization=True)
        
        # 유사도 검색 수행
        docs = vectorstore.similarity_search(query, k=3)
        
        # 결과를 문자열로 포맷
        results = []
        for i, doc in enumerate(docs, 1):
            results.append(f"검색 결과 {i}:\n{doc.page_content}\n")
        
        return "\n".join(results) if results else "관련 메뉴 정보를 찾을 수 없습니다."
        
    except Exception as e:
        return f"메뉴 검색 중 오류가 발생했습니다: {str(e)}"

print("3개의 도구 정의 완료")

# ChatOpenAI 모델 설정
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7,
    max_tokens=1000
)

# 3개의 도구를 LLM에 바인딩
tools = [tavily_search_func, wiki_summary, db_search_cafe_func]
llm_with_tools = llm.bind_tools(tools)

print("LLM에 도구 바인딩 완료")

3개의 도구 정의 완료
LLM에 도구 바인딩 완료


#### 3. 간단한 도구 호출 체인 구현체인 구조:

In [11]:
@chain
def cafe_assistant_chain(user_input: dict):
    """카페 메뉴 정보를 제공하는 AI 어시스턴트 체인
    
    체인 구조:
    사용자 질문 → LLM (도구 선택) → 도구 실행 → 결과 종합 → 최종 답변
    """
    
    question = user_input["question"]
    print(f"사용자 질문: {question}")
    
    # 1단계: LLM이 도구 선택 및 호출
    ai_msg = llm_with_tools.invoke([{"role": "user", "content": question}])
    
    # 2단계: 도구 호출 결과 수집
    tool_results = []
    
    if hasattr(ai_msg, 'tool_calls') and ai_msg.tool_calls:
        print(f"{len(ai_msg.tool_calls)}개의 도구가 호출됩니다:")
        
        for tool_call in ai_msg.tool_calls:
            tool_name = tool_call["name"]
            tool_args = tool_call["args"]
            
            print(f"   - {tool_name}: {tool_args}")
            
            # 각 도구별 조건부 실행 로직
            if tool_name == "tavily_search_func":
                result = tavily_search_func.invoke(tool_args)
            elif tool_name == "wiki_summary":
                result = wiki_summary.invoke(tool_args)
            elif tool_name == "db_search_cafe_func":
                result = db_search_cafe_func.invoke(tool_args)
            else:
                result = f"알 수 없는 도구: {tool_name}"
            
            tool_results.append({
                "tool": tool_name,
                "result": result
            })
    
    # 3단계: 도구 실행 결과를 최종 답변에 반영
    if tool_results:
        # 도구 실행 결과를 포함한 프롬프트 생성
        context = "\n\n".join([f"[{tr['tool']}] {tr['result']}" for tr in tool_results])
        
        final_prompt = f"""
사용자 질문: {question}

도구 실행 결과:
{context}

위 정보를 바탕으로 사용자의 질문에 정확하고 친절하게 답변해주세요.
메뉴 정보가 있다면 가격, 재료, 특징을 포함해서 답변해주세요.
"""
        
        final_response = llm.invoke([{"role": "user", "content": final_prompt}])
        return final_response.content
    else:
        # 도구 호출 없이 일반 답변
        return ai_msg.content

print("카페 어시스턴트 체인 구현 완료")

카페 어시스턴트 체인 구현 완료


#### 4. 테스트 질문 처리

In [12]:
def test_americano_question():
    """문제 5-1의 테스트 질문: 아메리카노의 가격과 특징"""
    
    print("\n" + "="*60)
    print("문제 5-1 테스트: 아메리카노 정보 검색")
    print("="*60)
    
    # 테스트 질문
    test_question = "아메리카노의 가격과 특징은 무엇인가요?"
    
    print(f"테스트 질문: {test_question}")
    print("-" * 50)
    
    try:
        # 예상 처리 과정:
        # 1. LLM이 질문 분석 → 메뉴 정보 필요 판단
        # 2. db_search_cafe_func 도구 호출
        # 3. 벡터 DB에서 "아메리카노" 관련 정보 검색
        # 4. 가격, 재료, 특징 정보 반환
        # 5. 정보를 자연어로 정리하여 사용자에게 답변
        
        response = cafe_assistant_chain.invoke({"question": test_question})
        
        print(f"AI 답변:")
        print(response)
        
        print("\n성공 기준 확인:")
        print("- 정확한 가격 정보 포함 여부")
        print("- 메뉴 특징 설명 포함 여부")
        print("- 자연스러운 답변 생성 여부")
        
    except Exception as e:
        print(f"테스트 실패: {e}")

def additional_tests():
    """추가 테스트 질문들"""
    
    additional_questions = [
        "카페인이 없는 음료 추천해주세요",
        "가장 비싼 메뉴는 무엇인가요?",
        "라떼 종류에는 어떤 것들이 있나요?",
        "커피의 역사에 대해 알려주세요"  # 위키피디아 도구 테스트용
    ]
    
    print("\n" + "="*60)
    print("추가 테스트 질문들")
    print("="*60)
    
    for i, question in enumerate(additional_questions, 1):
        print(f"\n 테스트 {i}: {question}")
        print("-" * 40)
        
        try:
            response = cafe_assistant_chain.invoke({"question": question})
            print(f"답변: {response}")
        except Exception as e:
            print(f"오류: {e}")
        
        print("-" * 40)

# 메인 실행 함수
def main():
    """메인 실행 함수"""
    print("카페 메뉴 도구 호출 체인 - 문제 5-1 시작")
    
    # 벡터 DB 존재 여부 확인
    if not os.path.exists("./db/cafe_db"):
        print("벡터 DB를 먼저 생성합니다...")
        try:
            vectorstore = create_vector_db()
        except Exception as e:
            print(f"벡터 DB 생성 실패: {e}")
            return
    else:
        print("기존 벡터 DB 발견")
    
    # 문제 5-1의 핵심 테스트 실행
    test_americano_question()
    
    # 추가 테스트 실행
    additional_tests()
    
    # 대화형 모드
    print("\n" + "="*60)
    print("대화형 모드 (종료하려면 'quit' 입력)")
    print("="*60)
    
    while True:
        user_question = input("\n 질문: ").strip()
        
        if user_question.lower() in ['quit', 'exit', '종료']:
            print("카페 어시스턴트를 종료합니다.")
            break
        
        if not user_question:
            continue
            
        try:
            response = cafe_assistant_chain.invoke({"question": user_question})
            print(f"\n 답변: {response}")
        except Exception as e:
            print(f"\n오류: {e}")

# 스크립트 실행
if __name__ == "__main__":
    main()

카페 메뉴 도구 호출 체인 - 문제 5-1 시작
기존 벡터 DB 발견

문제 5-1 테스트: 아메리카노 정보 검색
테스트 질문: 아메리카노의 가격과 특징은 무엇인가요?
--------------------------------------------------
사용자 질문: 아메리카노의 가격과 특징은 무엇인가요?
1개의 도구가 호출됩니다:
   - db_search_cafe_func: {'query': '아메리카노'}
AI 답변:
아메리카노는 가격이 ₩4,500이며, 주요 원료는 에스프레소와 뜨거운 물입니다. 이는 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피로, 원두 본연의 맛을 가장 잘 느낄 수 있고 깔끔하고 깊은 풍미가 특징입니다. 또한 설탕이나 시럽을 추가할 수도 있습니다. 아이스 아메리카노도 동일한 가격으로 판매되며, 차가운 물과 얼음이 추가되어 시원한 맛을 느낄 수 있는 제품입니다. 따라서 더운 날씨에는 인기가 높습니다.

성공 기준 확인:
- 정확한 가격 정보 포함 여부
- 메뉴 특징 설명 포함 여부
- 자연스러운 답변 생성 여부

추가 테스트 질문들

 테스트 1: 카페인이 없는 음료 추천해주세요
----------------------------------------
사용자 질문: 카페인이 없는 음료 추천해주세요
1개의 도구가 호출됩니다:
   - db_search_cafe_func: {'query': '카페인이 없는 음료'}
답변: 카페인이 없는 음료로는 콜드브루와 아이스 아메리카노를 추천드립니다. 

1. 콜드브루
   - 가격: ₩5,000
   - 주요 원료: 콜드브루 원액, 차가운 물
   - 특징: 부드럽고 달콤한 맛이 특징이며, 산미가 적어 누구나 부담 없이 즐길 수 있습니다. 얼음과 함께 시원하게 제공됩니다.

2. 아이스 아메리카노
   - 가격: ₩4,500
   - 주요 원료: 에스프레소, 차가운 물, 얼음
   - 특징: 깔끔하고 시원한 맛이 특징이며, 원두 본연의 풍미를 느낄 수 있습니다. 더운 날씨에 인기가

KeyboardInterrupt: Interrupted by user